In [15]:
import uproot
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib as mpl
mpl.rcParams['text.usetex'] = False



file_path = "c:/cygwin64/home/charl/25CERN/CassetteReception/ROOTFiles/summary_of_v3a_TTU_modules.root"
#input("Please enter the file path") 
#current path to test file! "c:/cygwin64/home/charl/25CERN/CassetteReception/summary_of_v3a_TTU_modules.root" 

In [16]:

file = uproot.open(file_path)
treenames = file.keys()
treenames = [t[:-2] for t in treenames]

tree_list = []
for treename in treenames:
    tree_list.append(f"{file_path}:{treename}")

module_df = pd.DataFrame(columns=['name', 'i_vs_v', 'adc_stdd', 'adc_mean'])
pdf_plots = PdfPages(f'{treenames[0]}_{treenames[-1][-4:]}plots.pdf')


In [17]:
dict_mods = {'module_name':[], 'meas_v':[], 'meas_i':[]}
i_v_list = []

for index_tree, branches in enumerate(uproot.iterate(tree_list)):
        mod_name = treenames[index_tree]    
        list_y = [(i*(10**6)) for i in branches["meas_i"].tolist()]
        #while list_y and list_y[-1] == 0:
        #        list_y.pop()
                        
        list_x = branches["meas_v"].tolist()[:len(list_y)] 
        
        print(len(list_x))

        dict_mods['module_name'].append(mod_name)
        dict_mods['meas_v'].append(list_x)
        dict_mods['meas_i'].append(list_y)

        dataframe = file[str(treenames[index_tree])].arrays(library = "pd")
        i_v_df= dataframe.drop(columns=["adc_stdd", "adc_mean"])
        i_v_df.to_csv(f"CSVFiles/{treenames[index_tree]}.csv", index = False)

235
235
235
235
235
235
235
235
235
235
235
235
235
235
235
235
235
235
235


In [28]:
dict_mods = {key: val for key, val in dict_mods.items() if val != 0}
i_vs_v_df = pd.DataFrame.from_dict(dict_mods)

for i in range(len(i_vs_v_df.index)): #i vs v plotting
        row = i_vs_v_df.index[i]
        
        plt.plot(i_vs_v_df['meas_v'].loc[row], i_vs_v_df['meas_i'].loc[row], marker = ".", linewidth = 1, markersize = 4, label = i_vs_v_df['module_name'].loc[row])
        plt.grid(which='minor', color="#D1D1D1", linestyle='-', axis = 'x')
        plt.grid(which='major', color="#454545", linestyle='-')
        plt.minorticks_on()
        plt.title("Current vs Voltage")
        plt.yscale("log")
        plt.xlabel("Voltage (V)")
        plt.ylabel("Current (μA)")
        plt.xlim([0,1000])
plt.legend(loc = 'lower right',bbox_to_anchor=(1.4,0))
plt.savefig(f"{treenames[0]}_{treenames[-1][-4:]}_i_v", bbox_inches="tight")
#pdf_plots.savefig()

#plt.show()
plt.close()


In [ ]:
x = []
y_mean = []
y_stdd = []
y_type = [y_mean,y_stdd]

for index_tree, branches in enumerate(uproot.iterate(tree_list)):
        #formatting data for plots and building csvs
        for idx, adc in enumerate(["adc_stdd", "adc_mean"]):
                adc_list = branches[adc]
                print(f"length of adc list: {len(adc_list)}, branches name: {branches}, adc: {adc}")
                #building csvs
                adc_df = pd.read_csv(f"CSVFiles/{treenames[index_tree]}.csv")
                #try:
                print("made it!")
                adc_df.insert(idx + 2, adc, adc_list)
                adc_df.to_csv(f"CSVFiles/{treenames[index_tree]}.csv", index = False)
                #except Exception as e:
                #        print("Warning: you are trying to make a file that already exits. Please delete any files that you wish to rewrite.", e)

                #formatting data
                for index in range(len(adc_df.index)-1):
                        if adc == "adc_stdd":
                                y_stdd.append(adc_df.iat[index, 2])
                        else:
                                y_mean.append(adc_df.iat[index, 3])
                                x.append(index)
                
        #plots
        fig, (ax_stdd, ax_mean) = plt.subplots(2, sharex=True)
        ax_stdd.set_ylim(-2,6)
        ax_mean.set_ylim(-2,6)
        ax_stdd.scatter(x, y_stdd)
        ax_mean.scatter(x, y_mean)

        ax_stdd.set_ylabel("adc_stdd (noise)")
        ax_mean.set_ylabel("adc_mean (pedetance)")

        ax_stdd.grid(visible=None, which='major', axis='both')
        ax_stdd.xaxis.set_ticks(np.arange(0, 250, 20))
        ax_mean.grid(visible=None, which='major', axis='both')
        ax_mean.xaxis.set_ticks(np.arange(0, 250, 20))
        ax_stdd.grid(visible=None, which='minor', axis='x', linestyle = '-')
        ax_mean.grid(visible=None, which='minor', axis='x', linestyle = '-')
        
        fig.suptitle(str(treenames[index_tree]))
        fig.set_figwidth(9)
        plt.xlim([0,240])
        #plt.figure(treenames[index_tree])
        #plt.savefig(f"{treenames[index_tree]}_adc.pdf")
        #pdf_plots.savefig()
        plt.close()
        #plt.show()

length of adc list: 235, branches name: [{meas_i: 4e-09, meas_v: 4.97e-05, adc_stdd: 19.2, adc_mean: 81.5}, ..., {...}], adc: adc_stdd
made it!
length of adc list: 235, branches name: [{meas_i: 4e-09, meas_v: 4.97e-05, adc_stdd: 19.2, adc_mean: 81.5}, ..., {...}], adc: adc_mean
made it!
length of adc list: 235, branches name: [{meas_i: 2.88e-08, meas_v: 3.5e-05, adc_stdd: 30.4, adc_mean: 67.8}, ...], adc: adc_stdd
made it!
length of adc list: 235, branches name: [{meas_i: 2.88e-08, meas_v: 3.5e-05, adc_stdd: 30.4, adc_mean: 67.8}, ...], adc: adc_mean
made it!
length of adc list: 235, branches name: [{meas_i: 2.07e-08, meas_v: 3.31e-05, adc_stdd: 20.7, adc_mean: 20.3}, ...], adc: adc_stdd
made it!
length of adc list: 235, branches name: [{meas_i: 2.07e-08, meas_v: 3.31e-05, adc_stdd: 20.7, adc_mean: 20.3}, ...], adc: adc_mean
made it!
length of adc list: 235, branches name: [{meas_i: 2e-08, meas_v: 4.65e-05, adc_stdd: 31.2, adc_mean: 72.6}, ..., {...}], adc: adc_stdd
made it!
length of 

In [20]:
print(adc_df)

           meas_i        meas_v     adc_stdd     adc_mean
0    1.035118e-08      0.000042   257.575300   353.848572
1    4.204217e-08     10.000200   236.487230   347.891754
2    4.969168e-08     20.000260   225.488080   332.544952
3    5.524811e-08     30.015460   230.217160   327.780518
4    6.296668e-08     40.026660   214.219590   325.980377
..            ...           ...          ...          ...
230  0.000000e+00  10000.000000    14.449750   132.177139
231  0.000000e+00  10000.000000    10.959539   141.135162
232  0.000000e+00  10000.000000     1.354642   665.105042
233  0.000000e+00  10000.000000     1.353091   670.755859
234  0.000000e+00  10000.000000  1001.000000  1001.000000

[235 rows x 4 columns]


STEP 2!!!!!!!

In [21]:
import pandas as pd
dataframe = file[str(treenames[0])].arrays(library = "pd")
dataframe = dataframe.drop(columns = ["adc_stdd", "adc_mean"])
#dataframe.to_csv("singleplot.csv", index = False)


#Currently specialized to the one practice plot lol


In [22]:
pd.read_csv("singleplot.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'singleplot.csv'